In [1]:
import psycopg2
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('form5500_data/config.json') as f:
   conf = json.load(f)
   host = conf['host']
   database = conf['database']
   user = conf['user']
   passw = conf['passw']

In [3]:
conn_str = "host={} dbname={} user={} password={}".format(host, database, user, passw)

In [4]:
conn = psycopg2.connect(conn_str)

<h3>Query - selected fields for 2017 only</h3>

In [5]:
query = "SELECT sb.SB_EIN, sb.SB_PN, sb.SB_TOT_PARTCP_CNT, sb.SB_EFF_INT_RATE_PRCNT, sb.SB_FNDNG_SHORT_IND, \
       sb.SB_PR_YR_FNDNG_PRCNT, sb.SB_PLAN_TYPE_CODE, \
       f.BUSINESS_CODE, f.SCH_SB_ATTACHED_IND, sb.SB_FNDNG_TGT_PRCNT \
FROM sb_full sb \
LEFT JOIN f5500_full f \
ON sb.SB_EIN = f.SPONS_DFE_EIN AND sb.SB_PN = f.SPONS_DFE_PN \
WHERE sb.SB_PLAN_YEAR_BEGIN_DATE BETWEEN '2017-01-01' AND '2017-12-31';"

In [6]:
eda_df = pd.read_sql(query, con=conn)

<h3>Check null count by column</h3>

In [7]:
print('Total records: {},\nNull distribution:\n{}'.format(len(eda_df), eda_df.isna().sum()))


Total records: 24273,
Null distribution:
sb_ein                       0
sb_pn                        0
sb_tot_partcp_cnt           29
sb_eff_int_rate_prcnt       35
sb_fndng_short_ind          39
sb_pr_yr_fndng_prcnt       181
sb_plan_type_code            0
business_code            15043
sch_sb_attached_ind      15082
sb_fndng_tgt_prcnt          20
dtype: int64


In [8]:
pd.notnull(eda_df).sum()

sb_ein                   24273
sb_pn                    24273
sb_tot_partcp_cnt        24244
sb_eff_int_rate_prcnt    24238
sb_fndng_short_ind       24234
sb_pr_yr_fndng_prcnt     24092
sb_plan_type_code        24273
business_code             9230
sch_sb_attached_ind       9191
sb_fndng_tgt_prcnt       24253
dtype: int64

<h3>Drop all nulls for now </h3>

In [9]:
eda_df2 = eda_df.dropna()

In [10]:
pd.notnull(eda_df2).sum()

sb_ein                   9113
sb_pn                    9113
sb_tot_partcp_cnt        9113
sb_eff_int_rate_prcnt    9113
sb_fndng_short_ind       9113
sb_pr_yr_fndng_prcnt     9113
sb_plan_type_code        9113
business_code            9113
sch_sb_attached_ind      9113
sb_fndng_tgt_prcnt       9113
dtype: int64

In [11]:
pd.notnull(eda_df2).sum()

sb_ein                   9113
sb_pn                    9113
sb_tot_partcp_cnt        9113
sb_eff_int_rate_prcnt    9113
sb_fndng_short_ind       9113
sb_pr_yr_fndng_prcnt     9113
sb_plan_type_code        9113
business_code            9113
sch_sb_attached_ind      9113
sb_fndng_tgt_prcnt       9113
dtype: int64

In [12]:
eda_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9113 entries, 19 to 24271
Data columns (total 10 columns):
sb_ein                   9113 non-null int64
sb_pn                    9113 non-null int64
sb_tot_partcp_cnt        9113 non-null object
sb_eff_int_rate_prcnt    9113 non-null float64
sb_fndng_short_ind       9113 non-null object
sb_pr_yr_fndng_prcnt     9113 non-null float64
sb_plan_type_code        9113 non-null object
business_code            9113 non-null object
sch_sb_attached_ind      9113 non-null object
sb_fndng_tgt_prcnt       9113 non-null float64
dtypes: float64(3), int64(2), object(5)
memory usage: 783.1+ KB


<h3>Create integer versions of feature columns</h3>

<h3>Participant count</h3>

In [13]:
eda_df2['sb_tot_partcp_cnt'] = pd.to_numeric(eda_df2['sb_tot_partcp_cnt'])

<h3>Prior year shortfall indicator</h3>

In [14]:
eda_df2['sb_fndng_short_ind'].unique()

array(['2', '1'], dtype=object)

In [15]:
eda_df2['sb_fndng_short_ind'] = pd.to_numeric(eda_df2['sb_fndng_short_ind'])

<h3>Create new column for left 2 digits of business code
<br>Make dummies</h3>


In [16]:
eda_df2['sector'] = pd.to_numeric(eda_df2['business_code'].str[:2])

In [17]:
eda_df2 = pd.concat([eda_df2, pd.get_dummies(eda_df2['sector'].values, prefix_sep='_')],axis=1)

In [18]:
eda_df2.drop('sch_sb_attached_ind', inplace=True, axis=1)
eda_df2.drop('sb_ein', inplace=True, axis=1)
eda_df2.drop('sb_pn', inplace=True, axis=1)
eda_df2.drop('business_code', inplace=True, axis=1)
eda_df2.drop('sb_plan_type_code', inplace=True, axis=1)

In [19]:
eda_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14871 entries, 0 to 24271
Data columns (total 30 columns):
sb_tot_partcp_cnt        9113 non-null float64
sb_eff_int_rate_prcnt    9113 non-null float64
sb_fndng_short_ind       9113 non-null float64
sb_pr_yr_fndng_prcnt     9113 non-null float64
sb_fndng_tgt_prcnt       9113 non-null float64
sector                   9113 non-null float64
11                       9113 non-null float64
21                       9113 non-null float64
22                       9113 non-null float64
23                       9113 non-null float64
31                       9113 non-null float64
32                       9113 non-null float64
33                       9113 non-null float64
42                       9113 non-null float64
44                       9113 non-null float64
45                       9113 non-null float64
48                       9113 non-null float64
49                       9113 non-null float64
51                       9113 non-null float6

<h3>Combine dummies and other features into X featureset</h3>

In [23]:
y_fs = eda_df2['sb_fndng_tgt_prcnt'].values

In [25]:
eda_df2.drop('sb_fndng_tgt_prcnt', inplace=True, axis=1)

In [ ]:
def clean_data(df, train=False):
    '''This is all the data "cleaning" for now...will revisit after 
       further EDA
    '''
    df_clean = df.dropna()
    
    #convert string values to integers where appropriate
    df_clean['sb_tot_partcp_cnt'] = pd.to_numeric(df_clean['sb_tot_partcp_cnt'])
    df_clean['sb_fndng_short_ind'] = pd.to_numeric(df_clean['sb_fndng_short_ind'])
    #drop un-needed columns
    df_clean.drop('sch_sb_attached_ind', inplace=True, axis=1)
    df_clean.drop('sb_ein', inplace=True, axis=1)
    df_clean.drop('sb_pn', inplace=True, axis=1)
    #create new column for sector
    df_clean['sector'] = pd.to_numeric(df_clean['business_code'].str[:2])
    
    if train:
        y=df_clean['sb_fndng_tgt_prcnt'].pop
        return df_clean, y
    else:
        return df_clean

<h1>Features EDA</h1>

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
X2 = prelim_feat_df[['ptp_cnt', 'sb_eff_int_rate_prcnt', 'sf_ind', \
                         'sb_pr_yr_fndng_prcnt', 'sb_fndng_tgt_prcnt', \
                          'sb_plan_type_code']]

In [ ]:
plt.style.use(['dark_background'])
scatter_matrix(X2, alpha=0.6, figsize=(12, 12), diagonal='hist', hist_kwds={'bins':100})
plt.show()

<h3>EIR - restrict to values between 1 and 10 (appear to be invalid data entries)</h3>

In [ ]:
X2[X2['ptp_cnt']<=100].describe()

In [ ]:
plt.hist(X2[(X2['ptp_cnt']<4000) & (X2['ptp_cnt']>100)]['ptp_cnt'],bins=100)
plt.title('Distribution of Participant Count (100 < PC < 4000)')
plt.show()

In [ ]:
eir_nonzero = X2[(X2['sb_eff_int_rate_prcnt'] >1) & (X2['sb_eff_int_rate_prcnt'] < 10) \
                & (X2['sb_fndng_tgt_prcnt'] > 0) & (X2['sb_fndng_tgt_prcnt'] < 400) & (X2['sb_plan_type_code'] == '1')]
y_eirnonzero = eir_nonzero['sb_fndng_tgt_prcnt'].values

In [ ]:
eir_nonzero.shape, y_eirnonzero.shape, eir_nonzero['sb_eff_int_rate_prcnt'].shape

In [ ]:
eir_nonzero['sb_eff_int_rate_prcnt'].describe()

In [ ]:
plt.hist(eir_nonzero['sb_eff_int_rate_prcnt'],bins=100)
plt.title('Distribution of EIR (1.0 < EIR < 10.0)')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
#plt.scatter(X['sb_pr_yr_fndng_prcnt'], y_fs)
plt.scatter(eir_nonzero['sb_eff_int_rate_prcnt'], y_eirnonzero)
plt.xlabel('Effective Interest Rate')
plt.ylabel('Funded Status (AVA basis)')
plt.title('FS vs EIR')
plt.show()

In [ ]:
pd.DataFrame(eir_nonzero['sb_eff_int_rate_prcnt']).boxplot(sym=)
plt.show()

In [ ]:
sns.boxplot(pd.DataFrame(eir_nonzero['sb_eff_int_rate_prcnt']))
plt.title('Boxplot of EIR')
plt.show()

In [ ]:
eir_nonzero['sb_eff_int_rate_prcnt'].describe()

In [ ]:
sns.boxplot(prelim_feat_df['sb_fndng_tgt_prcnt'])
plt.title('Boxplot of FS - all plans')
plt.show()

In [ ]:
sns.boxplot(eir_nonzero['sb_fndng_tgt_prcnt'])
plt.title('Boxplot of FS (0<FS<400%)')
plt.show()